In [1]:
from osgeo import gdal
import numpy as np
import matplotlib.pyplot as plt
import tables
import h5py

In [8]:
data=gdal.Open(r"E://LST project/AG100.v003.29.076.0001_delhi_west.h5")
print(gdal.Info(data))
hdf=gdal.Open(data.GetSubDatasets()[1][0],gdal.GA_ReadOnly)

Driver: HDF5/Hierarchical Data Format Release 5
Files: E://LST project/AG100.v003.29.076.0001_delhi_west.h5
Size is 512, 512
Coordinate System is `'
Metadata:
  ASTER_GDEM_ASTGDEM_Description=ASTER GDEM resampled to NAALSED
  Emissivity_Mean_Description=Mean Emissivity for each pixel on grid-box using all ASTER data from 2000-2010
  Emissivity_SDev_Description=Emissivity Standard Deviation for each pixel on grid-box using all ASTER data from 2000-2010
  Geolocation_Latitude_Description=Latitude
  Geolocation_Longitude_Description=Longitude
  Land_Water_Map_LWmap_Description=Land Water Map using ASTER visible bands
  NDVI_Mean_Description=Mean NDVI for each pixel on grid-box using all ASTER data from 2000-2010
  NDVI_SDev_Description=NDVI Standard Deviation for each pixel on grid-box using all ASTER data from 2000-2010
  Observations_NumObs_Description=Number of values used in computing mean and standard deviation for each pixel
  Temperature_Mean_Description=Mean Temperature (K) for ea

In [17]:

# h=gdal.Open("HDF5:E:\LST project\AG100.v003.20.072.0001.h5://Emissivity/Mean")
# o=h.GetRasterBand(1)
# emissivity=hdf.ReadAsArray()
# emissivity=emissivity[1,:,:]
# gdal.Translate(r"E:\\AST.tif",h,format='GTiff')

### for hdf5 to GTiff conversion
# astr=gdal.Translate(" ",h,format='GTiff')
# p=astr.GetRasterBand(1)
# p.ReadAsArray()

In [18]:
h=gdal.Open("HDF5:E://LST project/AG100.v003.29.076.0001_delhi_west.h5://Emissivity/Mean")
h.GetGCPProjection()

'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]]'

In [19]:
s=gdal.Open(r"E:\LST project\train_data_set\delhi\LE07_L1TP_146040_20010504_20170205_01_T1_sr_band4.tif")
proj=s.GetProjection()
proj

'PROJCS["WGS 84 / UTM zone 43N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",75],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AUTHORITY["EPSG","32643"]]'

In [20]:
projected=gdal.Warp(" ",h,
         dstSRS=proj,
         )
t=gdal.Warp(r"E:\LST project\train_data_set\delhi\ASTER_PROJ_west.tif",projected,
         xRes=100,yRes=100,
          resampleAlg=gdal.GRA_Average
         )

In [21]:
geo=t.GetGeoTransform()
proj=t.GetProjection()
print(geo,"\n\n",proj)

(597353.3502032508, 100.0, 0.0, 3209690.8176620584, 0.0, -100.0) 

 PROJCS["WGS 84 / UTM zone 43N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",75],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AUTHORITY["EPSG","32643"]]


In [22]:
AE3=t.ReadAsArray()[3]
AE4=t.ReadAsArray()[4]
AE3.shape

(1122, 993)

In [23]:
emissivity=0.0002147*AE3+0.0007789*AE4+0.0058
emissivity.shape

(1122, 993)

In [24]:
emissivity[emissivity<=0.7]=np.nan
emissivity

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [25]:
driver=gdal.GetDriverByName("GTiff")
ds=driver.Create(r"E:\LST project\train_data_set\delhi\ASTER_100m_Emissivity_L7_west.tif",AE3.shape[1],AE3.shape[0], 1,gdal.GDT_Float32)
ds.SetProjection(proj)
ds.SetGeoTransform(geo)
ds.GetRasterBand(1).WriteArray(emissivity)
ds=None